To find the path we are using OpenCV to detect the poles, and then Pathfinding library to find a path in the gird. Since it's challeging for computer to detect a whole pole, we will concentrate on detecting the white tops, and then calculate the base from there. After we get the X,Y coordinates of the base of the poles we'll overlay a virtual grid on top of the image, and calcualte the position of the poles in that grid. We will then use the grid positions to block out the grid cells that the robot can't pass through and use that as an input for path finding. 
 


In [5]:
#install open-cv
#https://pypi.org/project/opencv-python/
!pip install opencv-python

     |████████████████████████████████| 28.7MB 5.6kB/s eta 0:00:01     |████████████████▎               | 14.6MB 661kB/s eta 0:00:22     |████████████████████████        | 21.5MB 1.7MB/s eta 0:00:05████████████████████████▍     | 23.6MB 2.0MB/s eta 0:00:03████████████████████████▊     | 23.9MB 2.0MB/s eta 0:00:03��███████████████▌  | 26.4MB 3.3MB/s eta 0:00:01��███████████████████████████ | 27.8MB 138kB/s eta 0:00:07
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
#install pathfinding
#https://pypi.org/project/pathfinding/
!pip install pathfinding

You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
#OpenCV used for template matching
import cv2

import numpy as np
import math

# Pathfinding algorithms based on Pathfinding.JS for python 2 and 3.
# Using A*
# https://pypi.org/project/pathfinding/
from pathfinding.core.diagonal_movement import DiagonalMovement
from pathfinding.core.grid import Grid
from pathfinding.finder.a_star import AStarFinder
from pathfinding.finder.best_first import BestFirst
from pathfinding.finder.dijkstra import DijkstraFinder
from pathfinding.finder.bi_a_star import BiAStarFinder
from pathfinding.finder.ida_star import IDAStarFinder
from pathfinding.finder.breadth_first import BreadthFirstFinder

In [8]:
# Image to find the patterns/template in
img_rgb = cv2.imread('RobotUprising_still_high.png')

In [14]:
def find_base_coordinates():
    
    #Change image to gray to ignore color differences in pattern matching
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

    # Load the pole tip template image, with 0 color depth - gray scale image 
    template = cv2.imread('poletip3.png',0)
    # Get width and the height of the template
    w, h = template.shape[::-1]

    # Use OpenCV to match the template in the original image
    # This finds the occurrences of the template in the source image 
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)

    # Matching precision threshold - higher means a more precise match
    threshold = 0.85

    # Retrieve the x and y positions of the the matched pole tips
    xs1, ys1 = np.where( res >= threshold)

    # Do a second pass of pattern matching with another template because not all objects were detected in first pass
    # You can do multiple passes by different templates to increase the accuracy
    # template = cv2.imread('poletip4.png',0)
    # w, h = template.shape[::-1]
    # res = cv2.matchTemplate(img_gray,template,method)
    # threshold = 0.87
    # xs2, ys2 = np.where( res >= threshold)

    # If doing multiple passess, merge X and Y results
    # top_coords = (np.concatenate([xs1, xs2]), np.concatenate([ys1, ys2]))
    # otherwise just set xs1, xs2 into an tuple of arrays
    top_coords = (xs1, ys1)
    
    # Array to store coordinates of the base of the poles
    base_coords = []

    # Loop over all matches
    for pt_top in zip(*top_coords[::-1]):
        
        # Calculate the correction function
        
        # 100 = height of the pole at the top most point of the image minus the height of the lowest pole in the image (pole_heigh_delta)
        # 1600 = distance in pixels between lowest and highest pole 
        # 1768 = Y coordinates of the camera (position where the base of the pole and the top of the pole are the same)
        vertical_corr = int(100/1600 * (1768-pt_top[1]+170))
        
        # 40 = horizontal offset between the x position of the tip and x position of the base of the pole (e.g. if you draw a straight line from the top to the bottom of the pole, the number is the difference betwen these two lines)
        # 600 = distance between x coordinates of the camera and horizontal top position of the left most pole 
        # 825 = X coordinates of the camera (position where the base of the pole and the top of the pole are the same)
        horizontal_corr = int(40/600 * (825-pt_top[0]))

        # Here we are assuming that the height of the poles are similar in length
        
        # Get the coordinates of the base of the pole
        pt_base = (pt_top[0] + horizontal_corr, pt_top[1]+vertical_corr)

        # Store the coordinate in an array
        base_coords.append(pt_base)

        # To validate results you can 
        # draw rectangle for the top of the pole
        # cv2.rectangle(img_rgb, pt_top, (pt_top[0] + w, pt_top[1] + h), (0,0,255), 2)

        # draw rectangle for the base of the pole
        # cv2.rectangle(img_rgb, pt_base, (pt_base[0] + w, pt_base[1] + h + 0), (255,0,0), 2)

    # write the results to an image
    # cv2.imwrite(meth + '.png',img_rgb)

    # save the coordinate to disk if needed
    # np.save('base_coords', base_coords)

    return base_coords

In [10]:
# based on X and Y calculate a position in the grid where the blocking cell is
def get_blocking_cells(cell_size_x, cell_size_y, obstacle_coords):
    
    grid = []
    for coord in obstacle_coords:
        # translate x coord in cell-x
        cell_x =  math.floor(coord[0] / cell_size_x)
        # translate y coord in cell-y
        cell_y =  math.floor(coord[1] / cell_size_y)

        grid.append((cell_x, cell_y))
    return grid


In [11]:
# generate an overlay grid on top of the image
# we are placing a straight square grid, while slightly cropping the image
# the image could be rotated to compensate for the position of the camera
def generate_grids(blocking_cells, width, height, cell_size_x, cell_size_y):
    grids = []
    for x in range(0, math.floor(width / cell_size_x)+1):
        row = []
        for y in range(0, math.floor(height / cell_size_y)+1):
            row .append(1)
        grids.append(row)

    for (x,y) in blocking_cells:
        grids[x][y] = 0

    return grids


In [12]:
def find_path(matrix):
    g = Grid(matrix=matrix)
    start = g.node(1, 7)
    end = g.node(15, 7)
    finder = AStarFinder()
    path, runs = finder.find_path(start, end, g)
    print('operations:', runs, 'path length:', len(path))
    print(g.grid_str(path=path, start=start, end=end))


In [15]:
# the path is marked below with x, where s is the start of the path and e is the end of the path and # is position of the 

if __name__ == "__main__":
    corordinates =  find_base_coordinates()
    block_points = get_blocking_cells(100, 100, corordinates)
    grids = generate_grids(block_points, 1768, 1532, 100, 100)
    find_path(grids)
    #print(grids)

operations: 45 path length: 21
+----------------+
|         #      |
|          #     |
|    #           |
|   #  ##        |
|     ##     #   |
|    #   ##      |
|     #    #    #|
| sx  #  #xxxxxxe|
|  x## ##xx #  # |
|  xxx#xxx ##    |
|  # xxx#    #   |
|  #   #  #    # |
|    #   # ##    |
|   #  #      #  |
|     #          |
|          #     |
|                |
|                |
+----------------+
